In [0]:
# Read data from officeDataProject file and perform below analytics on it
# 1. print total number of employee in company
# 2. print total number of department in company
# 3. print total number of employees in each department
# 4. print total number of employees in each state
# 5. print total number of employees in each state in each department
# 6. print min and max salary in each department and sort it in ascending order
# 7. Print the number of employees working in NY state under finance department whose bonus are greater than average bonus of 
# employees in NY state
# 8. Raise the salary $500 of all employees whose age is greater than 45
# 9. create df of all those employees whose age is greater than 45 and save it.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit,udf,min,max,avg,sum,count,mean
spark = SparkSession.builder.appName('DF-Mini-Project').getOrCreate()
df = spark.read.options(header='true',inferSchema='true').csv('/FileStore/tables/OfficeDataProject.csv')
# print('Number of Employees in company:',count())
df.show()
df.cache()


+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
# 2. print total number of department in company
depDF = df.select('department').distinct()
print('Number of departments in company:',depDF.count())
depDF.show()

Number of departments in company: 6
+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [0]:
# 3. print total number of employees in each department
print('Total number of employees in each department:')
df.groupBy('department').count().show()

Total number of employees in each department:
+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [0]:
# 4. print total number of employees in each state
print('Total number of employees in each state:')
df.groupBy('state').count().show()

Total number of employees in each state:
+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



In [0]:
# 5. print total number of employees in each state in each department
df.groupBy('state','department').count().show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
|   CA|   Finance|   35|
|   CA|  Accounts|   35|
|   CA|Purchasing|   32|
|   WA|        HR|   47|
|   AK|Purchasing|   30|
|   WA|  Accounts|   27|
|   WA|Purchasing|   38|
|   AK|     Sales|   38|
|   AK|  Accounts|   37|
|   WA| Marketing|   39|
|   LA|        HR|   41|
|   LA|     Sales|   35|
|   AK|        HR|   25|
|   LA|   Finance|   29|
|   AK|   Finance|   37|
|   LA|Purchasing|   45|
+-----+----------+-----+
only showing top 20 rows



In [0]:
# 6. print min and max salary in each department and sort it in ascending order
df.groupBy('department').agg(min('salary').alias('Min Salary'),max('salary').alias('Max salary')).sort('Min Salary','Max salary').show()

+----------+----------+----------+
|department|Min Salary|Max salary|
+----------+----------+----------+
|   Finance|      1006|      9899|
|  Accounts|      1007|      9890|
|        HR|      1013|      9982|
| Marketing|      1031|      9974|
|     Sales|      1103|      9982|
|Purchasing|      1105|      9985|
+----------+----------+----------+



In [0]:
# 7. Print the number of employees working in NY state under finance department whose bonus are greater than average bonus of 
# employees in NY state
#Note: collect() function in df returns resulted data as list which we can save in variable and refer in dataframe
avgbonus = df.filter(col('state') == 'NY').groupBy('state').agg(avg('Bonus').alias('avg_bonus')).select('avg_bonus').collect()[0]['avg_bonus']
# type(avgbonus)
df.filter((col('department') == 'Finance') & (col('state') == 'NY') & (df.bonus > avgbonus)).count()


Out[113]: 17

In [0]:
df.createOrReplaceTempView('oficedata')

In [0]:
sprkdf = spark.sql("select count(*) as Total_Emp from oficedata where bonus > (select avg(bonus) as avg_bonus from oficedata group by state having state = 'NY') group by state,department having state = 'NY' and department = 'Finance'")
sprkdf.show()
type(sprkdf)

+---------+
|Total_Emp|
+---------+
|       17|
+---------+

Out[105]: pyspark.sql.dataframe.DataFrame

Out[97]: 162

In [0]:
# 8. Raise the salary $500 of all employees whose age is greater than 45
def rais(age,salary):
    if age > 45:
        return salary + 500
    else:
        return salary

raiseUDF = udf(lambda x,y:rais(x,y),IntegerType())
df.withColumn('salary',raiseUDF(df.age,df.salary)).show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  6217| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  4204| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [0]:
# 9. create df of all those employees whose age is greater than 45 and save it.
df45 = df.filter(col('age') > 45)
df45.write.csv('/FileStore/tables/OfficeData/above45/')

In [0]:
%fs ls /FileStore/tables/OfficeData/above45/

path,name,size,modificationTime
dbfs:/FileStore/tables/OfficeData/above45/_SUCCESS,_SUCCESS,0,1686313268000
dbfs:/FileStore/tables/OfficeData/above45/_committed_6918879319512289421,_committed_6918879319512289421,114,1686313268000
dbfs:/FileStore/tables/OfficeData/above45/_started_6918879319512289421,_started_6918879319512289421,0,1686313268000
dbfs:/FileStore/tables/OfficeData/above45/part-00000-tid-6918879319512289421-0e7d6e94-fc73-4c31-a252-8516b59d5b4e-1828-1-c000.csv,part-00000-tid-6918879319512289421-0e7d6e94-fc73-4c31-a252-8516b59d5b4e-1828-1-c000.csv,7104,1686313268000


In [0]:
%sql
select count(*) as Total_Emp from oficedata where bonus > (select avg(bonus) as avg_bonus from oficedata group by state having state = 'NY') group by state,department having state = 'NY' and department = 'Finance'

Total_Emp
17
